# Create Percentile Scores

Create the Percentile data, from scores cached in the Score notebook. 


In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

from demosearch import *
from pathlib import Path



In [67]:
# Setup the package and the cache 

pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
import  pylib 

def get_cache(pkg):
    return FileCache(Path(pkg.path).parent.joinpath( 'cache'))

pkg_root = Path(pkg.path).parent

cache = get_cache(pkg)

 

In [68]:
%%time
# Reads the files cached by the Score notebook and concat them into
# a dataset
frames = [ cache.get_df(f) for f in cache.list('business_scores/scores/')]
df = pd.concat(frames)

# All CBSA that have less than 150 records are grouped into a single
# category. 
t = df.groupby('cbsa').count().total_population.reset_index()
small_cbsa = t.loc[t.total_population<150].cbsa.to_list()
df['group_cbsa'] = df.cbsa.replace(small_cbsa, '31000US00000')

len(df), len(frames)

CPU times: user 21.7 s, sys: 1.05 s, total: 22.7 s
Wall time: 33.4 s


(496623, 5000)

In [69]:
df.head()

,total_population,male,female,over25_college,over25_high_school,seniors,households,households_unmaried,housing_owned_college,housing_rented_college,...,bar,cafe,active,travel,food,cbsa,naics,group,geometry,group_cbsa
0,15692.004947,8345.588060,7346.416887,2008.046067,7829.781693,2897.592292,5661.056290,386.154188,1024.773368,175.348180,...,3.271914,0.000000,1.170573,57.966400,1.422571,31000US34060,441120,auto,POINT (-79.97178 39.65603),31000US34060
1,20498.169260,10157.550981,10340.618279,3528.244892,6900.730943,1856.560945,6782.958559,400.579569,1684.088096,517.123114,...,0.000000,0.000000,2.636499,17.900014,1.403604,31000US21340,445210,food,POINT (-106.47724 31.77561),31000US21340
2,918626.987058,432778.913318,485848.073741,167465.128843,324924.235980,113063.453178,342645.961628,24809.111797,53281.619855,50515.087453,...,57.455629,51.189541,194.793388,2074.468229,156.488281,31000US37980,442110,shop,POINT (-75.15342 39.91441),31000US37980
3,288348.085439,141679.580102,146668.505337,50426.995053,117503.365435,47747.345063,102377.166581,7631.293479,22034.781129,7037.615514,...,7.738789,6.244411,27.763176,663.636364,5.038613,31000US38060,722511,ent,POINT (-111.76944 33.42932),31000US38060
4,69041.224410,34494.422690,34546.801720,11472.182702,23647.906270,8607.869637,23501.632873,1522.381126,5144.651377,1765.664115,...,0.000000,1.170573,1.208508,63.045658,0.487739,31000US26420,453220,shop,POINT (-95.44389 30.28791),31000US26420


In [6]:
demo_cols_l = ['total_population', 'male', 'female', 'over25_college',
       'over25_high_school', 'seniors', 'households', 'households_unmaried',
       'housing_owned_college', 'housing_rented_college', 'agg_income',
       'agg_hh_income']


osm_cols_l = ['primary', 'secondary', 'tertiary', 'trunk', 'highway',
       'entertain', 'restaurant', 'casual', 'shop', 'bar', 'cafe', 'active',
       'travel', 'food']
  
# Programatically separate point counts from demographic variables. Point counts
# are smaller, but more importantly, their std dec is much smaller, with a gap between the two groups from 
# about 800 to 11,000, so 5,000 is a good dividing line. 

stds = df.drop(columns=['naics']).describe().T.sort_values('std')
    
osm_cols = list(stds[stds['std']<5_000].index)
demo_cols = list(stds[stds['std']>5_000].index)
 
# Check that our programatic seperation works. But this will break when we change the columns in the census dataset 
assert set(demo_cols) == set(demo_cols_l)
assert set(osm_cols) == set(osm_cols_l)
              

In [49]:
# Create percentiles dataset

def make_pctile_df(df):
    """Create the percentiles dataset"""

    frames = []
    for idx, g in df.groupby('group_cbsa'):
        t = np.nanpercentile(g[demo_cols+osm_cols], np.linspace(0,100,101), axis=0)
        pct = pd.DataFrame(t, columns = demo_cols+osm_cols).round(2)
        pct['cbsa'] = idx
        pct.index.name = 'pct'
        frames.append(pct.set_index('cbsa',append=True))

    pct = pd.concat(frames) 
    
    return pct
 
def col_pctile(df, cbsa, col, value):
    """Use the CBSA percentiles map to find the percentile of a column value """
    t = df.loc[(slice(None), cbsa),:]
    # The sorting ensures that when there ae a lot of percentiles that have value 0, we
    # return the largest percentile value, rather than 0
    return (t[col]-value).abs().sort_values(ascending=False).idxmin()[0]
    
pct = make_pctile_df(df)
    
col_pctile(pct, '31000US31080', 'cafe', 21.0) # SHould be 50
    

50

In [63]:
df.to_csv('../data/all_scores.csv', index=False)
pct.to_csv('../data/percentiles.csv')

In [50]:
# Check that a sub-sample returns the same results. Can we use fewer records?
pct_t = make_pctile_df(df)

for (pct_v, cbsa), row in list(pct.sample(20).iterrows()):
    target_value = row['cafe']
    
    #pct_t = make_pctile_df(df.sample(200_000))
   
    v = col_pctile(pct, cbsa, 'cafe', target_value)
    
    print(f'{row.name} target={target_value} target_pct={pct_v} calc_pct={v}')
    
    

(23, '31000US20700') target=0.0 target_pct=23 calc_pct=11
(46, '31000US14140') target=0.0 target_pct=46 calc_pct=61
(56, '31000US26140') target=0.0 target_pct=56 calc_pct=51
(44, '31000US44420') target=2.28 target_pct=44 calc_pct=44
(64, '31000US35980') target=2.06 target_pct=64 calc_pct=57
(81, '31000US21340') target=0.0 target_pct=81 calc_pct=60
(3, '31000US48900') target=0.0 target_pct=3 calc_pct=9
(17, '31000US36100') target=0.0 target_pct=17 calc_pct=51
(46, '31000US44700') target=2.76 target_pct=46 calc_pct=46
(70, '31000US45900') target=14.39 target_pct=70 calc_pct=70
(92, '31000US33780') target=1.76 target_pct=92 calc_pct=93
(66, '31000US19100') target=7.83 target_pct=66 calc_pct=66
(50, '31000US31860') target=0.0 target_pct=50 calc_pct=61
(10, '31000US31140') target=0.0 target_pct=10 calc_pct=14
(66, '31000US45300') target=9.4 target_pct=66 calc_pct=66
(10, '31000US14020') target=0.0 target_pct=10 calc_pct=3
(100, '31000US36220') target=0.57 target_pct=100 calc_pct=99
(81, '31

pct,0,1,2,3,4
cbsa,31000US00000,31000US00000,31000US00000,31000US00000,31000US00000
households_unmaried,0.00,4.15,6.97,9.58,13.35
housing_owned_college,0.00,14.55,25.45,39.96,54.42
seniors,0.00,51.62,84.53,137.88,168.46
housing_rented_college,0.00,2.38,3.86,5.58,7.30
over25_high_school,9.01,140.97,219.32,323.24,411.61
over25_college,1.99,33.08,60.25,85.46,112.43
households,6.85,112.38,184.21,272.36,329.95
male,8.10,148.84,243.66,352.73,438.95
female,7.67,134.76,236.37,327.57,421.78
